# NiftyPET Example

This is a full demo of NiftyPET's default [OSEM](#OSEM "ordered subsets expectation maximisation") ($n_\text{max}=14$ subsets, span 11, Siemens Biograph mMR resolution), as well as a custom, explicit [MLEM](#MLEM "maximum likelihood expectation maximisation") incorporating [RM](#RM "resolution modelling").


Mathematically:

$$
{\bf y}^{(k+1)} = {{\bf y}^{(k)} \over \sum_n{{\bf H}^T{\bf X}_n^T{\bf A}^T{\bf N}^T{\bf 1}}}
    \circ
    \sum_n{ {\bf H}^T{\bf X}_n^T{\bf A}^T{\bf N}^T
        { {\bf m} \over {\bf NA}{\bf X}_n{\bf H}{\bf y}^{(k)} + {\bf r} + {\bf s} }
    },
$$

- $k$ is iteration number
- $H$ applies a Gaussian PSF
- $X_n$ is the system matrix for subset $n$ (MLEM has just one subset)
- $m, r, s$ are measured, randoms, and scatter

----

- Author: Casper O. da Costa-Luis [casper.dcl@{physics.org|ieee.org|ucl.ac.uk|kcl.ac.uk}](mailto:casper.dcl@physics.org)
- Date: 2019-21

----

# Imports

In [ ]:
# imports
%matplotlib notebook

import logging
from functools import partial
from os import path
from pathlib import Path

import cuvec as cu
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from tqdm.auto import trange

from niftypet import nipet, nimpa

logging.basicConfig(level=logging.INFO, format=nipet.LOG_FORMAT)
print(nipet.gpuinfo())
# get all the scanner parameters
mMRpars = nipet.get_mmrparams()
mMRpars['Cnt']['LOG'] = logging.INFO
# conversion for Gaussian sigma/[voxel] to FWHM/[mm]
SIGMA2FWHMmm = (8 * np.log(2))**0.5 * np.array([mMRpars['Cnt']['SO_VX' + i] for i in 'ZYX']) * 10
# image-space PSF function
imsmooth = partial(nimpa.imsmooth, Cnt=mMRpars['Cnt'])

# Load & Process Raw Data

In [ ]:
folderin = Path("amyloidPET_FBP_TP0")
# automatically categorise the input data
datain = nipet.classify_input(folderin, mMRpars)
# output path
opth = Path(datain['corepath']) / "niftyout"
# show categorised data
datain

In [ ]:
# hardware mu-map (bed, head/neck coils)
mu_h = nipet.hdw_mumap(datain, [1, 2, 4], mMRpars, outpath=opth, use_stored=True)

In [ ]:
# create histogram
mMRpars['Cnt']['BTP'] = 0
mSino = nipet.mmrhist(datain, mMRpars, outpath=opth, store=True, use_stored=True)
if False:                                                   # enable parametric bootstrap
    mMRpars['Cnt']['BTP'] = 2
    totCnt = 3e6                                            # total counts to simulate
    mMRpars['Cnt']['BTPRT'] = totCnt / mSino['psino'].sum() # count level ratio relative to original
    mSino = nipet.mmrhist(datain, mMRpars, outpath=opth / "BTP" / f"{totCnt:.3g}", store=True)

In [ ]:
# MR-based human mu-map
if True:                        # UTE-based object mu-map aligned (need UTE sequence or T1 for pseudo-CT)
    mu_o = nipet.align_mumap(
        datain,
        scanner_params=mMRpars,
        outpath=opth,
        store=True,
        use_stored=True,
        hst=mSino,
        t0=0,
        t1=0,                   # when both times are 0, will use full data
        itr=2,                  # number of iterations used for recon to which registering MR/UTE
        petopt='ac',            # what PET image to use (ac-just attenuation corrected)
        musrc='pct',            # source of mu-map (ute/pct)
        ute_name='UTE2',        # which UTE to use (UTE1/2 shorter/faster)
    )
else:                           # the same as above without any faff though (no alignment)
    mu_o = nipet.obj_mumap(datain, mMRpars, outpath=opth, store=True)

In [ ]:
try:
    mu = mu_h['im'] + mu_o['im'] # needs HW mu-maps
except (NameError, KeyError):
    mu = mu_o['im']
    mu_h = {'im': np.zeros_like(mu_o['im'])}

## Visualisations

In [ ]:
nimpa.imscroll(mu, titles=[r"$\mu$-map"], cmap='bone');

In [ ]:
# sinogram index (<127 for direct sinograms, >=127 for oblique sinograms)
nimpa.imscroll(
    {
        f"Prompt sinogram ({mSino['psino'].sum() / 1e6:.3g}M)": mSino['psino'],
        f"Delayed sinogram ({mSino['dsino'].sum() / 1e6:.3g}M)": mSino['dsino']}, cmap='inferno',
    fig=plt.figure(num=2, figsize=(9.5, 3.5)));

# Reconstruction

## OSEM

Note that since $\bf A$ and $\bf N$ are both diagonal matrix operators, the reconstruction equation can be slightly rewritten to reduce the number of calculations required per iteration:

$$
{\bf y}^{(k+1)} = {{\bf y}^{(k)} \over \sum_n{{\bf H}^T{\bf X}_n^T{\bf A}^T{\bf N}^T{\bf 1}}}
    \circ
    \sum_n{ {\bf H}^T{\bf X}_n^T
        { {\bf m} \over {\bf X}_n{\bf H}{\bf y}^{(k)} + ({\bf r} + {\bf s})/{({\bf A}^T{\bf N}^T {\bf 1})} }
    },
$$

In [ ]:
# setup
psf = nipet.prj.mmrrec.psf_config('measured', mMRpars['Cnt'])
msk = nipet.get_cylinder(mMRpars['Cnt'], rad=29., xo=0., yo=0., unival=1, gpu_dim=True) > 0.9

## Attenuation, Normalisation & Sensitivity
A = nipet.frwd_prj(mu, mMRpars, attenuation=True, dev_out=True) # imsmooth(mu, psf=psf)
N = nipet.mmrnorm.get_norm_sino(datain, mMRpars, mSino, gpu_dim=True)
AN = cu.asarray(A * N)

Sn = 14                   # number of subsets
sinoTIdx = [None] * Sn    # subset indices
sen = [None] * Sn         # sensitivity images for each subset
sen_inv_msk = [None] * Sn # masked inverse sensitivity image

for n in trange(Sn, unit="subset"):
    sinoTIdx[n] = cu.asarray(nipet.prj.mmrrec.get_subsets14(n, mMRpars)[0], 'int32')
    sen[n] = nipet.back_prj(nimpa.isub(AN, sinoTIdx[n]), mMRpars, isub=sinoTIdx[n], dev_out=True,
                            sync=False)
    imsmooth(sen[n], psf=psf, output_array=sen[n])
    assert sen[n].shape == (mMRpars['Cnt']['SZ_IMX'], mMRpars['Cnt']['SZ_IMY'],
                            mMRpars['Cnt']['SZ_IMZ'])
    sen_inv_msk[n] = cu.zeros_like(sen[n])
    sen_inv_msk[n][msk] = np.float32(1) / sen[n][msk]

In [ ]:
## Randoms

r = nipet.randoms(mSino, mMRpars)[0]
print("Randoms: %.3g%%" % (r.sum() / mSino['psino'].sum() * 100))

## Scatter

# Estimated image from two OSEM iterations (implicitly using voxel-driven scatter model)
eim = nipet.mmrchain(datain, mMRpars, mu_h=mu_h, mu_o=mu_o, itr=2, histo=mSino, outpath=opth)['im']
# Recalculate scatter
s = nipet.vsm(datain, (mu_h['im'], mu_o['im']), eim, mMRpars, histo=mSino, rsino=r)

# normalised randoms + scatter in GPU dimensions
r = nipet.mmraux.remgaps(r, mMRpars['txLUT'], mMRpars['Cnt'])
s = nipet.mmraux.remgaps(s, mMRpars['txLUT'], mMRpars['Cnt'])
rs_AN = nimpa.div(r + s, AN, default=1)
print("Scatter: %.3g%%" % (s.sum() / mSino['psino'].sum() * 100))

In [ ]:
y = cu.ones_like(sen[0])             # reconstructed image
Hy, XHy, crr, bim, mul = (None,) * 5 # temporary variables
m = cu.asarray(nipet.mmraux.remgaps(mSino['psino'], mMRpars['txLUT'], mMRpars['Cnt']))
rs_AN_sub = [nimpa.isub(rs_AN, idx) for idx in sinoTIdx]

for k in trange(4, desc="OSEM"):
    for n in trange(Sn, unit="subset", leave=False):
        # forward-project current reconstruction to sinogram space
        Hy = imsmooth(y, psf=psf, output_array=Hy, sync=False)
        XHy = nipet.frwd_prj(Hy, mMRpars, isub=sinoTIdx[n], attenuation=False, dev_out=True,
                             fullsino_out=False, output=XHy, sync=False)
        # add randoms and scatter estimates
        if False:
            # recalculate scatter
            s = nipet.vsm(datain, (mu_h['im'], mu_o['im']), y, mMRpars, histo=mSino, rsino=r)
            s = nipet.mmraux.remgaps(s, mMRpars['txLUT'], mMRpars['Cnt'])
            rs_AN = nimpa.div(nimpa.add(nimpa.isub(r, sinoTIdx[n]), nimpa.isub(s, sinoTIdx[n])),
                              nimpa.isub(AN, sinoTIdx[n]), default=1)
            XHy = nimpa.add(XHy, rs_AN, output=XHy, sync=False)
        else:
            XHy = nimpa.add(XHy, rs_AN_sub[n], output=XHy, sync=False)

        # measured sinogram subset
        crr = nimpa.isub(m, sinoTIdx[n], output=crr, sync=False)
        # corrections
        crr = nimpa.div(crr, XHy, default=1, output=crr, sync=False)
        # back-project corrections to image space
        bim = nipet.back_prj(crr, mMRpars, isub=sinoTIdx[n], dev_out=True, output=bim, sync=False)
        mul = imsmooth(bim, psf=psf, output_array=mul, sync=False)
        # apply FOV mask and normalise with scanner sensitivity
        mul = nimpa.mul(mul, sen_inv_msk[n], output=mul, sync=False)
        # update reconstructed image
        y = nimpa.mul(y, mul, output=y, sync=False)
cu.dev_sync()

In [ ]:
nimpa.imscroll(nipet.img.mmrimg.convert2e7(y, mMRpars['Cnt']), cmap='magma', vmin=0, vmax=np.percentile(y, 99.95));

## MLEM

In [ ]:
# setup
psf = nipet.prj.mmrrec.psf_config('measured', mMRpars['Cnt'])
msk = nipet.get_cylinder(mMRpars['Cnt'], rad=29., xo=0., yo=0., unival=1, gpu_dim=True) > 0.9

## Randoms

r = nipet.randoms(mSino, mMRpars)[0]
print("Randoms: %.3g%%" % (r.sum() / mSino['psino'].sum() * 100))

## Scatter

# Estimated image from two OSEM iterations (implicitly using voxel-driven scatter model)
eim = nipet.mmrchain(datain, mMRpars, mu_h=mu_h, mu_o=mu_o, itr=2, histo=mSino, outpath=opth)['im']
# Recalculate scatter
s = nipet.vsm(datain, (mu_h['im'], mu_o['im']), eim, mMRpars, histo=mSino, rsino=r)
print("Scatter: %.3g%%" % (s.sum() / mSino['psino'].sum() * 100))

# normalised randoms + scatter in GPU dimensions
r = nipet.mmraux.remgaps(r, mMRpars['txLUT'], mMRpars['Cnt'])
s = nipet.mmraux.remgaps(s, mMRpars['txLUT'], mMRpars['Cnt'])

## Attenuation, Normalisation & Sensitivity
A = nipet.frwd_prj(mu, mMRpars, attenuation=True, dev_out=True)
N = nipet.mmrnorm.get_norm_sino(datain, mMRpars, mSino, gpu_dim=True)
AN = cu.asarray(A * N)
rs_AN = nimpa.div(r + s, AN, default=1)

sim = nipet.back_prj(AN, mMRpars, dev_out=True, sync=False)
imsmooth(sim, psf=psf, output_array=sim)
sim_inv_msk = cu.zeros_like(sim)
sim_inv_msk[msk] = np.float32(1) / sim[msk]

In [ ]:
# "Attenuation": A, "Normalisation": N, "Scatter": s, "Randoms": r
nimpa.imscroll({"Prompts": mSino['psino'], "Delayed": mSino['dsino']}, cmap='inferno',
               fig=plt.figure(num=4, figsize=(9.5, 3.5)));

In [ ]:
## MLEM with RM
y = [np.ones_like(sim)]              # reconstructed image
Hy, XHy, crr, bim, mul = (None,) * 5 # temporary variables
m = cu.asarray(nipet.mmraux.remgaps(mSino['psino'], mMRpars['txLUT'], mMRpars['Cnt']))

for k in trange(4 * 14, desc="MLEM"):
    # forward-project current reconstruction to sinogram space
    Hy = imsmooth(y[-1], psf=psf, output_array=Hy, sync=False)
    XHy = nipet.frwd_prj(Hy, mMRpars, dev_out=True, fullsino_out=False, output=XHy, sync=False)
    # add randoms and scatter estimates
    if False:
        # recalculate scatter
        s = nipet.vsm(datain, (mu_h['im'], mu_o['im']), y[-1], mMRpars, histo=mSino, rsino=r)
        s = nipet.mmraux.remgaps(s, mMRpars['txLUT'], mMRpars['Cnt'])
        rs_AN = nimpa.div(r + s, AN, default=1)
    XHy = nimpa.add(XHy, rs_AN, output=XHy, sync=False)

    # corrections
    crr = nimpa.div(m, XHy, default=1, output=crr, sync=False)
    # back-project corrections to image space
    bim = nipet.back_prj(crr, mMRpars, dev_out=True, output=bim, sync=False)
    mul = imsmooth(bim, psf=psf, output_array=mul, sync=False)
    # apply FOV mask and normalise with scanner sensitivity
    mul = nimpa.mul(mul, sim_inv_msk, output=mul, sync=False)
    # update reconstructed image
    y.append(nimpa.mul(y[-1], mul, sync=False))

In [ ]:
# central slice across iterations
nimpa.imscroll(
    {
        f"{k}": nipet.img.mmrimg.convert2e7(y[k], mMRpars['Cnt'])[:, 90:-100, 110:-110]
        for k in range(7, len(y), 7)}, cmap='magma', vmin=0, vmax=np.percentile(y[-1], 99.95),
    fig=plt.figure(num=6, figsize=(9.5, 3.5)));